In [1]:
from RecoilNet import RecoilNet
from plot_model import plot_hist

from keras.optimizers import Adam, SGD
from loader_bot_omega import LoaderBot   # dynamic full image augmentation

import keras
import time
import json
import numpy as np

/home/brian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def run():
    # data_link_dict = get_skfold_data(path="../data/imgs/*.jpg")
    start_time = time.time()

    # decommisioned because inflight data augmentation solves a lot of these
    # problems

    # Use json to load the permanent dictionary that has been Created
    with open("../data/data_splits.json") as infile:
        data_link_dict = json.load(infile)

    EPOCHS = 10
    AUGMENTATION = 1    # could do 3 epochs of 10 augmentation or 30 of 1 which
                        # provides more data for plots to work with

    MINITRAINS = 10
    DO = 0.55  # drop out

    # for Adam inital LR of 0.0001 is a good starting point
    # for SGD initial LR of 0.001 is a good starting point
    LR = 0.0005
    DECAY = 0.5e-6
    L2_REG = 0.01
    OPTIMIZER = Adam(lr=LR, decay=DECAY)
    # OPTIMIZER = SGD(lr=LR, momentum=0.9, nesterov=True)

    MODEL_ID = 'v3_0a'

    plot_file = "model_{:}.png".format(MODEL_ID)
    weights_file = "weights/model_{:}_weights.h5".format(MODEL_ID)
    history_file = "histories/history_{:}.json".format(MODEL_ID)

    # # user parameters for LoaderBot v1.0
    # # Parameters for Generators
    # params = {'dim': (299, 299),
    #           'batch_size': 256,
    #           'n_classes': 128,
    #           'n_channels': 3,
    #           'shuffle': False}

    # These parameters are for LoaderBot v2.0
    # Parameters for Generators
    params = {'dim': (299, 299),
              'batch_size': 128,
              'n_classes': 128,
              'n_channels': 3,
              'augmentation': AUGMENTATION,
              'shuffle': True}
    #
    # Parameters for Generators
    test_params = {'dim': (299, 299),
                   'batch_size': 128,
                   'n_classes': 128,
                   'n_channels': 3,
                   'augmentation': 1,
                   'augment': False,
                   'shuffle': False}

    # Datasets
    X_train_img_paths = data_link_dict["X_test_1"]
    y_train = np.array(data_link_dict["y_test_1"])
    y_train = keras.utils.to_categorical(y_train - 1, num_classes=128).reshape(-1, 1)
    
    
    X_test_img_paths = data_link_dict["X_test_2"]
    y_test = np.array(data_link_dict["y_test_2"])
    y_test = keras.utils.to_categorical(y_test - 1, num_classes=128).reshape(-1, 1)
    
#     y_train = np.array(y_train).reshape(-1, 1)
#     y_test = np.array(y_test).reshape(-1,1)

    # Generators
    training_generator = LoaderBot(X_train_img_paths, y_train, **params)
    validation_generator = LoaderBot(X_test_img_paths, y_test, **test_params)

# #     setup model
#     base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
#     model = regular_brian_layers(base_model, 128, DO, l1_reg=0.0001)

    recoil = RecoilNet(summary=False)
    model = recoil.get_model()
    
    model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # print(model.summary())

    # mini-train 1, like normal
    # transfer learning
#     setup_to_transfer_learn(model, base_model, OPTIMIZER)

    history = {}   # preinitialize history log

    for mt in range(MINITRAINS):
        temp = mt + 1

        if temp == 1:
            # Run model
            new_history = model.fit_generator(generator=training_generator,
                                             validation_data=validation_generator,
                                             epochs=EPOCHS,
                                             use_multiprocessing=False)

            history["acc"] = new_history.history["acc"]
            history["val_acc"] = new_history.history["val_acc"]
            history["loss"] = new_history.history["loss"]
            history["val_loss"] = new_history.history["val_loss"]

        else:
            # mini-train 2
            OPTIMIZER = Adam(lr=LR / 2**temp, decay=DECAY)
            # try to fine tune some of the InceptionV3 layers also
#             setup_to_finetune(model, NB_IV3_LAYERS_TO_FREEZE - (5 * temp), OPTIMIZER, L2_REG)

            print("\n\n        Starting epoch {:}\n\n".format(EPOCHS * mt + 1))

            # Run model
            new_history = model.fit_generator(generator=training_generator,
                                              validation_data=validation_generator,
                                              epochs=EPOCHS,
                                              use_multiprocessing=False)

            # save the weights in case we want to predict on them later
            model.save(weights_file)

            history["acc"] += new_history.history["acc"]
            history["val_acc"] += new_history.history["val_acc"]
            history["loss"] += new_history.history["loss"]
            history["val_loss"] += new_history.history["val_loss"]

        # seems to be prepending a 0 to the list so ignore that
        history["acc"] = history["acc"]
        history["val_acc"] = history["val_acc"]
        history["loss"] = history["loss"]
        history["val_loss"] = history["val_loss"]

        plot_hist(history, plot_file, epochs=len(history["acc"]), sprint=True)

    # try to save the history so models can be more easily compared and Also
    # to better log results if going back is needed
    with open(history_file, "w") as outfile:
        json.dump(history, outfile)

    print("\n\n\n\nCompleted in {:6.2f} hrs".format(((time.time() - start_time)) / 3600))  # convert to hours


In [3]:
run()

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[128,256,75,75] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: batch_normalization_4/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NHWC", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_35/BiasAdd, batch_normalization_4/gamma/read, batch_normalization_4/beta/read, batch_normalization_47/Const_4, batch_normalization_47/Const_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss/mul/_10453 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_76987_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to se